In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.lite.python.interpreter import Interpreter

# Define Paths
dataset_path = 'Car-Bike-NonVehicle-Dataset'
model_path = "vehicle_classification_model.tflite"

def create_generators():
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2  # 80% train, 20% validation
    )
    
    train_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    
    validation_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, validation_generator

# Load Data
train_generator, validation_generator = create_generators()
num_classes = len(train_generator.class_indices)
print("Updated Class Indices:", train_generator.class_indices)  # Ensure "No Vehicle" is included

# Load Pretrained MobileNetV2
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

# Build Model
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Convert to TFLite
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tflite_converter.convert()
with open(model_path, 'wb') as f:
    f.write(tflite_model)

# Prediction Function with "No Vehicle" Handling
def predict_vehicle(image_path, confidence_threshold=0.7):
    interpreter = Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (150, 150))
    image = np.expand_dims(image.astype(np.float32) / 255.0, axis=0)
    
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    
    predicted_class = np.argmax(output[0])
    confidence = output[0][predicted_class]

    class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Ensure correct mapping

    # Apply confidence threshold: if below, return "No Vehicle"
    if confidence < confidence_threshold:
        return f"Prediction: No Vehicle (Low Confidence: {confidence:.2f})"
    
    return f"Prediction: {class_labels[predicted_class]} (Confidence: {confidence:.2f})"


Found 39200 images belonging to 3 classes.
Found 9799 images belonging to 3 classes.
Updated Class Indices: {'Bike': 0, 'Car': 1, 'Not_Vehicle': 2}


C:\Users\H P\AppData\Local\Temp\ipykernel_9904\1140944963.py:49: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
c:\Users\H P\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 299/1225 ━━━━━━━━━━━━━━━━━━━━ 15:19 993ms/step - accuracy: 0.9574 - loss: 0.2516

c:\Users\H P\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1225/1225 ━━━━━━━━━━━━━━━━━━━━ 1524s 1s/step - accuracy: 0.9803 - loss: 0.1011 - val_accuracy: 0.9972 - val_loss: 0.0086
Epoch 2/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 1116s 912ms/step - accuracy: 0.9946 - loss: 0.0180 - val_accuracy: 0.9941 - val_loss: 0.0158
Epoch 3/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 1191s 972ms/step - accuracy: 0.9943 - loss: 0.0175 - val_accuracy: 0.9964 - val_loss: 0.0109
Epoch 4/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 1177s 961ms/step - accuracy: 0.9950 - loss: 0.0145 - val_accuracy: 0.9939 - val_loss: 0.0203
Epoch 5/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 689s 563ms/step - accuracy: 0.9949 - loss: 0.0152 - val_accuracy: 0.9961 - val_loss: 0.0104
Epoch 6/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 627s 512ms/step - accuracy: 0.9949 - loss: 0.0143 - val_accuracy: 0.9972 - val_loss: 0.0084
Epoch 7/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 623s 508ms/step - accuracy: 0.9952 - loss: 0.0164 - val_accuracy: 0.9958 - val_loss: 0.0145
Epoch 8/10
1225/1225 ━━━━━━━━━━━━━━━━━━━━ 628s 512ms/step - accuracy: 0.

INFO:tensorflow:Assets written to: C:\Users\HP12D5~1\AppData\Local\Temp\tmprumx5kkg\assets


Saved artifact at 'C:\Users\HP12D5~1\AppData\Local\Temp\tmprumx5kkg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1804993208848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993210000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993211920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993211536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993210384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993212112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993211152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993209616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993212304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1804993210192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  18